# The Google of Quotes

## Assignment

The file “quotes.txt” contains pairs of lines, with the first line being a quote and the following line being the person who said it. We want to build a search engine that, given a word or words, finds the best matching quotes.  

(a) Build a list of full quotes (5 points). Read in the file, and create a list of full quotes of the form “quote - speaker”. For example, “The heart has its reasons, of which the mind knows nothing. - Blaise Pascal”.

(b) Words from full quotes (5 points). Write a function that takes a full quote as argument and outputs a list of the words in the list. The words should all be lower-case, and should contain only characters, digits, or underscore. Hint: Use the lower() function of strings, and re.split() to split into words, but you must figure out the regular expression for re.split().

(c) Build the postings-list dictionary (6 points). A postings-list is a dictionary whose keys are full quotes, and whose values are themselves dictionaries with key being a word, and value being the number of times the word occurs in the full quote. So, for the key “The heart has its reasons, of which the mind knows nothing. - Blaise Pascal”, the value will itself be the following dictionary: {’the’:2, ’heart’:1, ’has’:1, ’its’:1, ’reasons’:1, ’of’:1, ’which’:1, ’mind’:1, ’knows’:1, ’nothing’:1, ’blaise’:1, ’pascal’:1}.  

(d) Build the reverse postings-list dictionary (6 points). A reverse postings-list is a dictionary whose keys are the words, and the values are themselves dictionaries with the key being a full quote, and the value being the number of times the word appeared in the full quote. So, for the key “entertainer”, the value is the dictionary {’An actor is at most a poet and at least an entertainer. - Marlon Brando’: 1} (only this quote contains the word “entertainer”).  

(e) Write a TF-IDF function (8 points). To measure how much a full quote is about a particular word, one typically uses the TF-IDF measure.

  • TF stands for “term frequency”; the term frequency of a word w in a full quote q is the number of times w occurs in q, divided by the maximum number of times any word occurs in q.  
  • IDF stands for “inverse document frequency”: the IDF of a word w is the logarithm of the ratio of the total number N of full quotes to the number of full quotes in that contain the word w.  
  • TF-IDF of a word w for a full quote q is just the product of the TF and IDF.  
So, for the word “entertainer” in the Marlon Brando quote of part (d):  
  • The TF is 0.5 (it occurs once, while the most frequent word in that quote is “at”, which occurs twice, so the TF ratio is 0.5)  
  • The IDF is log (895/1), since there are 895 documents and the word “entertainer ” occurs in only one full quote. Write a function to compute the TF-IDF of any word in any full quote, using the postings and reverse-postings. Hint: Do import math and use math.log() to get logarithms.  

(f) Quote search using a single word (5 points). Write a function that takes a word as argument, and returns a dictionary whose keys are full quotes containing that word, and whose values are the TF-IDF score of that word for that full quote.  

(g) Quote search using multiple words (5 points). Write a function that takes a list of words as argument, and returns a dictionary whose keys are full quotes containing that word, and whose values are the sum of TF-IDF scores of all the words for that full quote.

## Approach


In [13]:
import copy
import re
from collections import Counter
from pprint import pprint
import math
from collections import defaultdict
import operator

### Part A
Part A required a list of each quote with it's appropriate author. However, the text file has a quote on one line and the author on the following line. In order to get each list item into the form "quote - author", two separate lists were created from the text file. The first list was a list of all of the quotes (taking in only odd line numbers). The second list was a list of all of the authors (taking in only even line numbers). To combine these lists, the join function was utilized, taking the an item from the quote list and pairing it with the appropriate item from the author list. The final list (full_quote_list) was returned from the function. 

In [ ]:
file_name = 'quotes.txt' #file located in jupyter library

quote_list = []
author_list = []
full_quote_list = []

def build_quote_list(textFile):
    #part a - build list of quotes-speaker
    
    count = 0
    
    with open(file_name) as textFile: #open the file as a text file
        for line in textFile:
            count += 1
            if count %2 != 0:
                quote_list.append(line.rstrip())
            else:
                author_list.append(line.rstrip())
    for i in range(len(quote_list)):
        full_quote_list.append(quote_list[i] + " - " + author_list[i])
    
    #the comments below were code for testing - commented out to eliminate output space
    #print full_quote_list
    #print len(quote_list)
    #print len(author_list)
    #print quote_list
    #print author_list
    
    return full_quote_list
    
build_quote_list('quotes.txt') 

### Part B
Working from the full_quote_list that was completed in part A, a second list needed to be created where each list item was a list of the words from a particular quote - excluding all special characters (-, ", ;, :, etc.). To accomplish this, a dummy list (quote_list_nochar) was used that deleted all special characters and converted all of the words into lower case. The new list (quote_list_word_split) took each word from the dummy list and split them into individual list items. The final list (quote_list_word_split) is returned and contains lists of words from each quote. 

In [ ]:
quote_list_word_split = []

def build_word_list(fullQuoteList):
    #part b - convert each word to a list item
    quote_list_nochar = copy.copy(fullQuoteList)
    
    for i in range(len(fullQuoteList)):
        quote_list_nochar[i] = quote_list_nochar[i].replace(".", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace(",", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace("?", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace(")", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace("(", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace('"', "")
        quote_list_nochar[i] = quote_list_nochar[i].replace(":", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace(";", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace("'", "")
        quote_list_nochar[i] = quote_list_nochar[i].replace("- ", "")
        quote_list_nochar[i] = quote_list_nochar[i].lower()
        
    for i in range(len(quote_list_nochar)):
        quote_list_word_split.append(re.split(" ", quote_list_nochar[i]))
    
    return quote_list_word_split

build_word_list(full_quote_list)


### Part C
The third part of this assignment asked for a new dictionary that listed the quote as the key and a dictionary of each word from the quote paired with the number of times it appears in the quote. (example: {quote1:{word1:3, word2:2, word3:1 ...}, quote2:{word1:3, word2:2, word3:1 ...}, ...}. To accomplish this, a blank dictionary titled "posting_list" was created. Next, each list item (which is a list itself) in the "quote_list_word_split" (compiled in part B) was read. After this, each word in that list was read and compared to see if it was in a dummy dictionary (word_list - initialized as blank). If the word did not exist in the dummy dictionary it was added with value being 1. If the word did exist, the value was incremented by 1. Finally, the posting_list dictionary was appended with the quote as the key (from full_quote_list) and the dummy word list dictionary as the value. 

In [ ]:
posting_list = {}

def posting_list_dictionary(quoteListWordSplit):
    #part c - create th" dictionary with the number of times a word appears in a quote
    
    for i in range(len(quoteListWordSplit)):
        word_list = {}
        for word in quoteListWordSplit[i]:
            if word in word_list:
                word_list[word] += 1
            else:
                word_list[word] = 1
        posting_list[full_quote_list[i]] = word_list
    
    return posting_list

posting_list_dictionary(quote_list_word_split)

### Part D

The fourth part of the assignment required a dictionary that listed each word in the list of quotes as the key, with the value being another dictionary of the quote the word it appears in as another key and the number of times the word appears in the quote as the value. In order to accomplish this, a list of all of the words (without any repeats was defined. Next, a for loop ran through each word in the word list, searched for it in the the postings list above, and made note of it's appearances. 

In [ ]:
word_dict = {}

def reverse_posting_dictionary(postingList):
    #part d - create a reverse posting of the dictionary

    word_list = []
    keys = posting_list.keys()
    
    for x in range(len(posting_list)):
        for y in quote_list_word_split[x]:
            if y in word_list and y != " " and y != "":
                pass
            else:
                word_list.append(y)
    #return word_list
    
    word_list = filter(None, word_list)
    
    #word_dict = {}
    
    for word in word_list:
        word_dict[word] = {}
        for quote in postingList:
            if word in postingList[quote]:
                counter = posting_list[quote][word]
                word_dict[word][quote] = counter
    
    
    return word_dict
    

reverse_posting_dictionary(posting_list) 
    

### Part E

Part E of the assignment involves creating a TD_IDF function for particular words. To accomplish this, a new dictionary was created to store each of the TD_IDF values. A word list was initialized using the method from part D. Next the function searched through each word in the word list and initializes the particular TD_IDF_List dictionary for that key. The word is searched for in each quote and the appropriate values for TD and IDF are gathered. Finally, the function returns the full list of TD_IDF values in the form {word: {quote1:td_idf value, quote2:td_idf value, ...} ...}

In [ ]:
td_idf_list = {}

def td_idf(postingList, reversePostingList):
    
    word_list = []
    
    for x in range(len(posting_list)):
        for y in quote_list_word_split[x]:
            if y in word_list and y != " " and y != "":
                pass
            else:
                word_list.append(y)
    
    word_list = filter(None, word_list)
    
    #return word_list
    
    quoteCount = len(full_quote_list)
    print quoteCount
    
    for word in word_list:
        td_idf_list[word]={}
        for quote in postingList:
            if word in postingList[quote]:
                maxWord = max(postingList[quote], key=postingList[quote].get)
                maxWord = postingList[quote][maxWord]
                td = (reversePostingList[word][quote])/maxWord
                idf = math.log(quoteCount/float(len(reversePostingList[word])))
                td_idf = td * idf
                td_idf_list[word][quote] = td_idf
                #print maxWord
    return td_idf_list
        
td_idf(posting_list, word_dict)    

### Part F

This part of the assignment asks for a function that can search on a single word and return the list of quotes the word appears in. To accomplish this, the function takes in the td_idf_list created above. It asks the user for a word to search on and determines if the word is in the td_idf_list (organized by word as the key and the dictionary of quotes and td-idf values as the value). If so, the function returns the list of quotes and td-idf values. If not an error message appears. 

In [38]:
def quote_search_single():
    searchWord = raw_input("What word would you like to search for? ")
    searchWord = searchWord.lower()
    
    if searchWord in td_idf_list:
        print
        print "Quote results for", searchWord, ":"
        print
        pprint(td_idf_list[searchWord])
    else:
        print
        print "I'm sorry, that word isn't in the list of quotes!"
        
quote_search_single()

What word would you like to search for? beautiful

Quote results for beautiful :

{'Being is desirable because it is identical with Beauty, and Beauty is loved because it is Being. We ourselves possess Beauty when we are true to our own being; ugliness is in going over to another order; knowing ourselves, we are beautiful; in self-ignorance, we are ugly. - Ambrose Bierce': 0.0,
 'In life, as in art, the beautiful moves in curves. - Edward Bulwer-Lytton': 0.0,
 'In the Orthodox spiritual tradition, the ultimate moral question we ask is the following: Is what we are doing, is what I am doing, beautiful or not? - Carolyn Gifford': 0.0,
 'Is there anything more beautiful than a beautiful, beautiful flamingo, flying across in front of a beautiful sunset? And he`s carrying a beautiful rose in his beak, and also he`s carrying a very beautiful painting with his feet. And also, you`re drunk. - Jack Handey': 4.840806801549768,
 'Love is the delightful interval between meeting a beautiful girl an

### Part G

In order to be able to search multiple words, a function was created that takes in a string of undetermined length. The function then converts the string into a list of individual words. Finally, the function searches each word and appends the quote_list dictionary with the quotes that contain those words and their td-idf value. 

In [48]:
def quote_search_multiple():
    searchWords = raw_input("What would you like to search for? ")
    searchWords = searchWords.lower()
    
    word_list = searchWords.replace(","," ").replace(";", " ").split()
    quote_list = {}
    #print word_list
    
    for word in word_list:
        if word in td_idf_list:
            quote_list[word] = td_idf_list[word]
    
    print
    print "Your results are: "
    print
    pprint(quote_list)
    print
    
quote_search_multiple()

What would you like to search for? beautiful, religion

Your results are: 

{'beautiful': {'Being is desirable because it is identical with Beauty, and Beauty is loved because it is Being. We ourselves possess Beauty when we are true to our own being; ugliness is in going over to another order; knowing ourselves, we are beautiful; in self-ignorance, we are ugly. - Ambrose Bierce': 0.0,
               'In life, as in art, the beautiful moves in curves. - Edward Bulwer-Lytton': 0.0,
               'In the Orthodox spiritual tradition, the ultimate moral question we ask is the following: Is what we are doing, is what I am doing, beautiful or not? - Carolyn Gifford': 0.0,
               'Is there anything more beautiful than a beautiful, beautiful flamingo, flying across in front of a beautiful sunset? And he`s carrying a beautiful rose in his beak, and also he`s carrying a very beautiful painting with his feet. And also, you`re drunk. - Jack Handey': 4.840806801549768,
               'Lov